In [ ]:
immutable Vertex
    id::Int
end

In [ ]:
type Vertices
    list::Vector{Vertex}
    num_valid::Int
    
    Vertices(n::Int) = new([Vertex(i) for i in 1:n], n)
end

In [ ]:
Base.length(v::Vertices) = v.num_valid
Base.getindex(v::Vertices, i::Int) = Base.getindex(v.list, i)

function reduce_vertices!(v::Vertices)
    v.num_valid -= 1
end

In [ ]:
immutable Edge
    tail::Vertex
    head::Vertex
end

In [ ]:
type Edges
    list::Vector{Edge}
    num_valid::Int
    
    Edges() = new(Edge[], 0)
end

In [ ]:
Base.start(v::Edges) = 1
Base.done(v::Edges, state::Int) = v.num_valid < state
Base.next(v::Edges, state) = (v.list[state], state + 1)
Base.length(v::Edges) = v.num_valid
Base.getindex(v::Edges, i::Int) = Base.getindex(v.list, i)

function Base.rand(v::Edges)
    return v.list[Base.rand(1:v.num_valid)]
end

In [ ]:
add_edge!(edges::Edges, tail::Vertex, head::Vertex) = add_edge!(edges, Edge(tail, head))
function add_edge!(edges::Edges, edge::Edge)
    edges.num_valid += 1

    if edges.num_valid > length(edges.list)
        push!(edges.list, edge)
    else
        edges.list[edges.num_valid] = edge
    end 
end

function remove_edge!(edges::Edges, index::Int)
    edges.list[index] = edges.list[edges.num_valid]
    edges.num_valid -= 1
    
end

function replace_edge!(edges::Edges, index::Int, new_edge::Edge)
    edges.list[index] = new_edge
end

In [ ]:
immutable Graph
    vertices::Vertices
    edges::Edges
    
    Graph(num_vertices::Int) = new(Vertices(num_vertices), Edges())
end

In [ ]:
function add_edges!(g::Graph, tail::Vertex, heads::Vector{Vertex})
    for head in heads
        add_edge!(g.edges, tail, head)
    end
end

function merge_vertices!(g::Graph, source::Vertex, target::Vertex)
    i = 1
    while i ≤ length(g.edges)
        edge = g.edges[i]
        
        if edge.tail == source
            if edge.head == target
                remove_edge!(g.edges, i)
                continue
            else
                replace_edge!(g.edges, i, Edge(target, edge.head))
            end
        elseif edge.head == source
            if edge.tail == target
                remove_edge!(g.edges, i)
                continue
            else
                replace_edge!(g.edges, i, Edge(edge.tail, target))
            end
        end
        
        i += 1
    end
    reduce_vertices!(g.vertices)
end

In [ ]:
immutable Element
    value::Int
    marker::Int
    rank::Int
end
    
Element(value::Int, rank::Int = 0) = Element(value, value, rank)

In [ ]:
init_elements(n::Int) = [Element(i) for i in 1:n]

function find!(v::Vector{Element}, i::Int)
    if v[i].marker != i
        v[i] = Element(i, find!(v, v[i].marker), v[i].rank)
    end
    return v[i].marker
end

function unite!(v::Vector{Element}, i_1::Int, i_2::Int)
    i_1_root = find!(v, i_1)
    i_2_root = find!(v, i_2)
    
    if v[i_1_root].rank < v[i_2_root].rank
        v[i_1_root] = Element(i_1_root, i_2_root, v[i_1_root].rank)
    elseif v[i_2_root].rank < v[i_1_root].rank
        v[i_2_root] = Element(i_2_root, i_1_root, v[i_2_root].rank)
    else
        v[i_2_root] = Element(i_2_root, i_1_root, v[i_2_root].rank)
        v[i_1_root] = Element(i_1_root, i_1_root, v[i_1_root].rank + 1)
    end
end

In [ ]:
get_tail!(e::Edge, v::Vector{Element}) = find!(v, e.tail.id)
get_head!(e::Edge, v::Vector{Element}) = find!(v, e.head.id)

is_looping!(e::Edge, v::Vector{Element}) = (get_tail!(e, v) == get_head!(e, v))

In [ ]:
function load_graph(filename)
    lines = open(readlines, filename, "r")
    g = Graph(length(lines))
    
    for line in lines
        v_ids = [parse(Int, s) for s in matchall(r"\d+", line)]
        vertices = [g.vertices[id] for id in v_ids if id ≥ v_ids[1]]
        tail = vertices[1]
        heads = vertices[2:end]

        add_edges!(g, tail, heads)
    end
    
    return g
end

In [ ]:
function get_karger_min_cut(g_original::Graph)
    n = length(g.vertices)
    min_cut = n
    for rep in 1:(3 * n^2 * ceil(log(n)))
        cut = randomized_contraction(g)
        (min_cut > cut) && (min_cut = cut)
    end
    
    return min_cut
end

In [ ]:
function randomized_contraction(g::Graph)
    v = init_elements(length(g.vertices))
    n = length(g.vertices)
    while n > 2
        e = rand(g.edges)
        tail = get_tail!(e, v)
        head = get_head!(e, v)
        if tail == head
            continue
        end
        unite!(v, tail, head)
        n -= 1
    end
    
    cut = 0
    for e in g.edges
        tail = get_tail!(e, v)
        head = get_head!(e, v)
        if tail != head
            cut += 1
        end
    end
    
    return cut
end

Correct answer is 17.

This code took Jupyter 50 sec to calculate the answer with $3 n^2 \lceil \log n \rceil = 720,000$ repetitions.
The speedup is due to using the union-find data structure.

In [48]:
FILE_PATH = "kargerMinCut.txt"
g = load_graph(FILE_PATH)
@time get_karger_min_cut(g)

 49.858841 seconds (4.36 M allocations: 3.381 GB, 0.61% gc time)


17